## Import dependencies

In [2]:
import gurobipy as gp
from gurobipy import GRB

## Preparate data

In [ ]:

# --- 1. Data Preparation ---
subjects = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic", "Genetique"]
weights = {"Anatomie": 8, "Biologie": 7, "Chirurgie": 7, "Diagnostic": 6, "Epidemiologie": 6, "Forensic": 5, "Genetique": 6}

# Candidate grades (Xavier vs Yvonne)
grades_x = {"Anatomie": 85, "Biologie": 81, "Chirurgie": 71, "Diagnostic": 69, "Epidemiologie": 75, "Forensic": 81, "Genetique": 88}
grades_y = {"Anatomie": 81, "Biologie": 81, "Chirurgie": 75, "Diagnostic": 63, "Epidemiologie": 67, "Forensic": 88, "Genetique": 95}

## Compute pros, cons and deltas

In [3]:
# Calculate Contributions (deltas)
deltas = {}
pros = []
cons = []

print("--- Contributions ---")
for s in subjects:
    # Contribution = weight * (grade_x - grade_y)
    diff = grades_x[s] - grades_y[s]
    contrib = weights[s] * diff
    deltas[s] = contrib
    
    if contrib > 0:
        pros.append(s)
    elif contrib < 0:
        cons.append(s)
    print(f"{s}: {contrib}")

print(f"\nPros: {pros}")
print(f"Cons: {cons}")

--- Contributions ---
Anatomie: 32
Biologie: 0
Chirurgie: -28
Diagnostic: 36
Epidemiologie: 48
Forensic: -35
Genetique: -42

Pros: ['Anatomie', 'Diagnostic', 'Epidemiologie']
Cons: ['Chirurgie', 'Forensic', 'Genetique']
